# Bokeh Application Development

There are many ways to share the plots that `bokeh` produces. This notebook introduces a development process that demonstrates some of these methods.

This notebook is based on the **crossfilter** demo application, available [here](https://github.com/bokeh/bokeh/blob/master/examples/app/crossfilter/main.py)

#### Notebook Setup

*As always, we import the packages needed at the top of the notebook.*

In [17]:
import pandas as pd
import numpy as np

from bokeh.layouts import row, column
from bokeh.models import Select, ColumnDataSource, ColorBar
from bokeh.palettes import Spectral5
from bokeh.plotting import curdoc, figure

from bokeh.io import output_notebook, show

In [18]:
output_notebook()

Loading BokehJS ...

### Load Data

In this case we are going to use set of sampled ata provided by `bokeh`.

In [19]:
from bokeh.sampledata.autompg import autompg_clean as data
df = data

In [24]:
df.head(2)

,mpg,cyl,displ,hp,weight,accel,yr,origin,mfr
0,18.0,8,307.0,130,3504,12.0,70,North America,chevrolet
1,15.0,8,350.0,165,3693,11.5,70,North America,buick


In [25]:
df.describe()

,mpg,displ,hp,weight,accel
count,392.000000,392.000000,392.000000,392.000000,392.000000
mean,23.445918,194.411990,104.469388,2977.584184,15.541327
std,7.805007,104.644004,38.491160,849.402560,2.758864
min,9.000000,68.000000,46.000000,1613.000000,8.000000
25%,17.000000,105.000000,75.000000,2225.250000,13.775000
50%,22.750000,151.000000,93.500000,2803.500000,15.500000
75%,29.000000,275.750000,126.000000,3614.750000,17.025000
max,46.600000,455.000000,230.000000,5140.000000,24.800000


In [26]:
df.dtypes

mpg       float64
cyl        object
displ     float64
hp          int64
weight      int64
accel     float64
yr         object
origin     object
mfr        object
dtype: object

### Data Cleaning & Preparation

In [27]:
df.cyl = df.cyl.astype(str)
df.yr = df.yr.astype(str)
del df['name']

KeyError: 'name'

In [30]:
columns = sorted(df.columns)
discrete = [x for x in columns if df[x].dtype == object]
continuous = [x for x in columns if x not in discrete]

In [32]:
columns

['accel', 'cyl', 'displ', 'hp', 'mfr', 'mpg', 'origin', 'weight', 'yr']

In [33]:
discrete

['cyl', 'mfr', 'origin', 'yr']

In [35]:
continuous

['accel', 'displ', 'hp', 'mpg', 'weight']

### Plot Function

In [36]:
def create_figure(source, x_sel, y_sel, size_sel=None, color_sel=None):
    """A basic figure creation function.
    We will need to move a copy of this function into the bokeh application we develop.
    """
    fig = figure(plot_height=600, plot_width=800, name='test_figure')
    fig.circle(x_sel, y_sel, source=source)
    return fig

In [ ]:
df_source = ColumnDataSource

In [10]:
# show(create_figure(df_source, 'mpg', 'cyl'))

### Application Development in Notebooks

From tutorial 11:

> The easiest way to embed a Bokeh application in a notebook is to make a function modify_doc(doc) that creates Bokeh content, and adds it to the document. This function can be passed to show, and the app defined by the function will be displayed inline.

In [37]:
def modify_doc(doc):
    
    # Convert the pandas dataframe to a native, bokeh object, a ColumnDataSource.
    df_source = ColumnDataSource(df)    
    
    def _create_figure(source, x_sel, y_sel):
        """A basic figure creation function."""
        fig = figure(plot_height=600, plot_width=800, name='Main_Figure')
        fig.circle(x_sel, y_sel, source=source)
        return fig
    
    # Define user selection controls.
    x_selector     = Select(title='X-Axis', value='mpg',  options=columns)
    y_selector     = Select(title='Y-Axis', value='hp',   options=columns)
    # Assign them to a list for ease of referencing.
    controllers = [x_selector, y_selector]
    
    # Define update callback for these selectors.
    def _callback(attr, old, new):
        """Callback function to run when the user changes a selector value.
        `attr`, `old` and `new` are required arguments for bokeh callback functions."""
        # We can select the old figure to replace by its name.
        fig_layout = doc.get_model_by_name('main_layout')
        fig_layout.children[-1] = _create_figure(df_source, x_selector.value, y_selector.value)
        # print(attr, old, new)
        
    # Add the callback to those selectors.
    for item in controllers:
        item.on_change('value', _callback)
        
    # Define a layout of the plot and controls.
    controls = column(controllers, width=300)
    layout = row(controls, _create_figure(df_source, x_selector.value, y_selector.value),
                 name='main_layout')
    
    # Add the layout to the `doc`.
    doc.add_root(layout)

In [38]:
show(modify_doc)

ERROR:bokeh.server.views.ws:Refusing websocket connection from Origin 'http://localhost:8889';                       use --allow-websocket-origin=localhost:8889 to permit this; currently we allow origins {'localhost:8888'}


In [15]:
from bokeh.transform import linear_cmap, factor_cmap, LinearColorMapper

In [17]:
help(linear_cmap)

Help on function linear_cmap in module bokeh.transform:

linear_cmap(field_name, palette, low, high, low_color=None, high_color=None, nan_color='gray')
    Create a ``DataSpec`` dict to apply a client-side ``LinearColorMapper``
    transformation to a ``ColumnDataSource`` column.
    
    Args:
        field_name (str) : a field name to configure ``DataSpec`` with
    
        palette (seq[color]) : a list of colors to use for colormapping
    
        low (float) : a minimum value of the range to map into the palette.
            Values below this are clamped to ``low``.
    
        high (float) : a maximum value of the range to map into the palette.
            Values above this are clamped to ``high``.
    
        low_color (color, optional) : color to be used if data is lower than
            ``low`` value. If None, values lower than ``low`` are mapped to the
            first color in the palette. (default: None)
    
        high_color (color, optional) : color to be used if da

In [18]:
def modify_doc_v2(doc):
    
    df_source = ColumnDataSource(df)    
    
    def _create_figure(source, x_sel, y_sel, size_sel='None', color_sel='None'):
        """A basic figure creation function."""
        fig = figure(plot_height=600, plot_width=800, name='Main_Figure')
        
        if color_sel is not 'None':
            start = min(df[color_sel])
            end =   max(df[color_sel])
            color = linear_cmap(color_sel, 'Viridis256', start, end)
        else: color = "#31AADE"
            
        fig.circle(x_sel, y_sel, source=source, color=color)
        return fig
    
    # Define user selection controls.
    x_selector     = Select(title='X-Axis', value='mpg',  options=columns)
    y_selector     = Select(title='Y-Axis', value='hp',   options=columns)
    size_selector  = Select(title='Size',   value='None', options=['None'] + continuous)
    color_selector = Select(title='Color',  value='None', options=['None'] + continuous)
    # Assign them to a list for ease of referencing.
    controllers = [x_selector, y_selector, size_selector, color_selector]
    
    # Define update callback for these selectors.
    def _callback(attr, old, new):
        """Callback function to run when the user changes a selector value.
        `attr`, `old` and `new` are required arguments for bokeh callback functions."""
        # We can select the old figure to replace by its name.
        fig_layout = doc.get_model_by_name('main_layout')
        fig_layout.children[-1] = _create_figure(df_source, x_selector.value,
                                                             y_selector.value,
                                                             size_selector.value,
                                                             color_selector.value)
        # print(attr, old, new)
        
    # Add the callback to those selectors.
    for item in controllers:
        item.on_change('value', _callback)
        
    # Define a layout of the plot and controls.
    controls = column(controllers, width=300)
    layout = row(controls, 
                 _create_figure(df_source, x_selector.value, y_selector.value,
                                size_selector.value, color_selector.value),
                 name='main_layout')
    
    # Add the layout to the `doc`.
    doc.add_root(layout)

In [20]:
show(modify_doc_v2)

### Theme the Application

*We want those nice dark colors like in the documentation demo.*

The yaml file contains:

```
attrs:
    Figure:
        background_fill_color: '#2F2F2F'
        border_fill_color: '#2F2F2F'
        outline_line_color: '#444444'
    Axis:
        axis_line_color: "white"
        axis_label_text_color: "white"
        major_label_text_color: "white"
        major_tick_line_color: "white"
        minor_tick_line_color: "white"
        minor_tick_line_color: "white"
    Grid:
        grid_line_dash: [6, 4]
        grid_line_alpha: .3
    Title:
text_color: "white"
```

In [21]:
def modify_doc_v3(doc):
    
    df_source = ColumnDataSource(df)    
    
    def _create_figure(source, x_sel, y_sel, size_sel='None', color_sel='None'):
        """A basic figure creation function."""
        fig = figure(plot_height=600, plot_width=800, name='Main_Figure')
        
        if color_sel is not 'None':
            start = min(df[color_sel])
            end =   max(df[color_sel])
            color_mapper = LinearColorMapper(palette='Viridis256', low=start, high=end)
            color_bar = ColorBar(color_mapper=color_mapper)
            color = {'field': color_sel, 'transform': color_mapper}
            fig.add_layout(color_bar, 'right')
        else: color = "#31AADE"
            
        fig.circle(x_sel, y_sel, source=source, color=color)
        
        # Apply styling.
        fig.background_fill_color = '#2F2F2F'
        fig.border_fill_color = '#2F2F2F'
        fig.outline_line_color = '#444444'
        
        fig.axis.axis_line_color = 'white'
        fig.axis.axis_label_text_color = 'white'
        fig.axis.major_label_text_color = 'white'
        fig.axis.major_tick_line_color = 'white'
        fig.axis.minor_tick_line_color = 'white'
        fig.axis.minor_tick_line_color = 'white'
        
        fig.grid.grid_line_dash = [6, 4]
        fig.grid.grid_line_alpha = 0.3
        
        return fig
    
    # Define user selection controls.
    x_selector     = Select(title='X-Axis', value='mpg',  options=columns)
    y_selector     = Select(title='Y-Axis', value='hp',   options=columns)
    size_selector  = Select(title='Size',   value='None', options=['None'] + continuous)
    color_selector = Select(title='Color',  value='None', options=['None'] + continuous)
    # Assign them to a list for ease of referencing.
    controllers = [x_selector, y_selector, size_selector, color_selector]
    
    # Define update callback for these selectors.
    def _callback(attr, old, new):
        """Callback function to run when the user changes a selector value.
        `attr`, `old` and `new` are required arguments for bokeh callback functions."""
        # We can select the old figure to replace by its name.
        fig_layout = doc.get_model_by_name('main_layout')
        fig_layout.children[-1] = _create_figure(df_source, x_selector.value,
                                                             y_selector.value,
                                                             size_selector.value,
                                                             color_selector.value)
        # print(attr, old, new)
        
    # Add the callback to those selectors.
    for item in controllers:
        item.on_change('value', _callback)
        
    # Define a layout of the plot and controls.
    controls = column(controllers, width=300)
    layout = row(controls, 
                 _create_figure(df_source, x_selector.value, y_selector.value,
                                size_selector.value, color_selector.value),
                 name='main_layout')
    
    # Add the layout to the `doc`.
    doc.add_root(layout)

In [22]:
show(modify_doc_v3)